In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_dir = "C:/Users/HP/Downloads/archive (8)/train"
test_dir = "C:/Users/HP/Downloads/archive (8)/test"
train_csv = "Training_set.csv"
test_csv = "Testing_set.csv"

In [4]:
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [13]:
# Ensure proper file paths
train_df['filepath'] = train_df['filepath'].apply(lambda x: os.path.normpath(x))
test_df['filepath'] = test_df['filepath'].apply(lambda x: os.path.normpath(x))


In [6]:
label_encoder = LabelEncoder()
train_df['label_encoded'] = label_encoder.fit_transform(train_df['label'])

In [9]:
IMG_SIZE = 128
def load_images(df):
    images = []
    labels = []
    for _, row in df.iterrows():
        img_path = row['filepath']
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0  # Normalize
        images.append(img)
        labels.append(row['label_encoded'])
    return np.array(images), np.array(labels)

In [12]:
print(train_df.head())

      filename                     label  \
0  Image_1.jpg          SOUTHERN DOGFACE   
1  Image_2.jpg                    ADONIS   
2  Image_3.jpg            BROWN SIPROETA   
3  Image_4.jpg                   MONARCH   
4  Image_5.jpg  GREEN CELLED CATTLEHEART   

                                            filepath  label_encoded  
0  C:/Users/HP/Downloads/archive (8)/train\Image_...             66  
1  C:/Users/HP/Downloads/archive (8)/train\Image_...              0  
2  C:/Users/HP/Downloads/archive (8)/train\Image_...             12  
3  C:/Users/HP/Downloads/archive (8)/train\Image_...             44  
4  C:/Users/HP/Downloads/archive (8)/train\Image_...             33  


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Define paths
train_dir = "path/to/train/"
test_dir = "path/to/test/"
train_csv = "path/to/Training_set.csv"
test_csv = "path/to/Testing_set.csv"

# Load CSV files
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

# Extract file names and labels (Assuming 'label' column exists)
train_df['filepath'] = train_dir + train_df['filename']
test_df['filepath'] = test_dir + test_df['filename']

# Encode labels
label_encoder = LabelEncoder()
train_df['label_encoded'] = label_encoder.fit_transform(train_df['label'])

# Load and preprocess images
IMG_SIZE = 128

def load_images(df):
    images = []
    labels = []
    for _, row in df.iterrows():
        img_path = row['filepath']
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0  # Normalize
        images.append(img)
        labels.append(row['label_encoded'])
    return np.array(images), np.array(labels)

X_train, y_train = load_images(train_df)
X_test, y_test = load_images(test_df)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

# Convert labels to categorical
num_classes = len(label_encoder.classes_)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# CNN Model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=30,
                    validation_data=(X_val, y_val),
                    verbose=1)

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"\nTest accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))

# Confusion Matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
